In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import ky_utils.ky_interpretable_functions as interpret
import ky_utils.ky_riskslim as slim
import ky_utils.ky_stumps as stumps

from pprint import pprint
from riskslim.helper_functions import load_data_from_csv, print_model
from sklearn.metrics import roc_auc_score

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\ky-fl-combined\ky-model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/KY-cleaned.csv").sort_values('person_id')
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/FL-cleaned.csv").sort_values('person_id')

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['misdemeanor_two_year'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['misdemeanor_two_year'].values

In [3]:
#### CART
depth = [5,6,7,8,9,10]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, seed=816)

### EBM
estimators = [40]
depth = [2]
learning_rate = [0.3]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, seed=816)

In [4]:
np.mean(cart_summary['FL_score']), np.mean(ebm_summary['FL_score'])

(0.5586146272855134, 0.5654148557046625)

### Lasso Stumps

In [5]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/KY-stumps.csv").sort_values('person_id')
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/FL-stumps.csv").sort_values('person_id')

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['misdemeanor_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['misdemeanor_two_year'].values

## columns 
cols = KY_X.columns[1:]

In [8]:
single_stump_model = stumps.stump_model(KY_X, KY_Y, FL_X, FL_Y, 0.002, cols, 816)
single_stump_model['features']

['age_at_current_charge20',
 'age_at_current_charge21',
 'age_at_current_charge24',
 'age_at_current_charge27',
 'age_at_current_charge36',
 'age_at_current_charge40',
 'age_at_current_charge43',
 'age_at_current_charge47',
 'p_arrest1',
 'p_arrest2',
 'p_arrest3',
 'p_arrest4',
 'p_arrest5',
 'p_arrest6',
 'p_charges1',
 'p_violence1',
 'p_felony1',
 'p_felony2',
 'p_felony3',
 'p_misdemeanor2',
 'p_misdemeanor3',
 'p_misdeassult1',
 'p_traffic1',
 'p_fta_two_year1',
 'p_fta_two_year2',
 'p_pending_charge1',
 'p_probation1',
 'p_incarceration1',
 'six_month1',
 'one_year1']

In [9]:
c_grid={'C': [0.001, 0.002]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [10]:
stumps_summary['best_params'], np.mean(stumps_summary['FL_score']), np.mean(stumps_summary['auc_diff'])

([{'C': 0.002}, {'C': 0.002}, {'C': 0.002}, {'C': 0.002}, {'C': 0.002}],
 0.5503488415130654,
 0.0004125800019412651)

### RiskSLIM

In [11]:
## train on best param chosen by Lasso Stumps from above
single_stump_model = stumps.stump_model(KY_X, 
                                        KY_Y, 
                                        FL_X, 
                                        FL_Y, 
                                        0.0008, 
                                        cols, 
                                        816)
selected_features = single_stump_model['features']
len(selected_features)

13

In [12]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_KY_X.insert(0, '(Intercept)', 1)

In [13]:
riskslim_summary = slim.risk_cv(sub_KY_X, 
                                KY_Y, 
                                sub_FL_X, 
                                FL_Y, 
                                y_label = 'misdemeanor_two_year', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime=1000, 
                                max_offset = 100,
                                c=1e-6, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
02/15/20 @ 08:05 PM | 183 rows in lookup table
02/15/20 @ 08:05 PM | ------------------------------------------------------------
02/15/20 @ 08:05 PM | runnning initialization procedure
02/15/20 @ 08:05 PM | ------------------------------------------------------------
02/15/20 @ 08:05 PM | CPA produced 2 cuts
02/15/20 @ 08:05 PM | running naive rounding on 3 solutions
02/15/20 @ 08:05 PM | best objective value: 0.6931
02/15/20 @ 08:05 PM | rounding produced 3 integer solutions
02/15/20 @ 08:05 PM | best objective value is 0.6931
02/15/20 @ 08:05 PM | running sequential rounding on 3 solutions
02/15/20 @ 08:05 PM | best objective value: 0.6931
02/15/20 @ 08:05 PM | sequential rounding produced 0 integer solutions
02/15/20 @ 08:05 PM | polishing 3 solutions
02/15/20 @ 08:05 PM | best objective value: 0.6931
02/15/20 @ 08:05 PM | polishing produced 3 integer solutions
02/15/20 @ 08:05 PM | initialization produced 5 feasible soluti

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/15/20 @ 08:05 PM | adding 221 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4256.
Tried aggregator 1 time.
Reduced MIP has 28 rows, 30 columns, and 81 nonzeros.
Reduced MIP has 13 binaries, 15 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/15/20 @ 08:06 PM | adding 228 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4251.
Tried aggregator 1 time.
Reduced MIP has 28 rows, 30 columns, and 81 nonzeros.
Reduced MIP has 13 binaries, 15 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/15/20 @ 08:06 PM | adding 223 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4255.
Tried aggregator 1 time.
Reduced MIP has 28 rows, 30 columns, and 81 nonzeros.
Reduced MIP has 13 binaries, 15 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Elapsed time = 33.30 sec. (40625.39 ticks, tree = 1.67 MB, solutions = 10)

User cuts applied:  366

Root node processing (before b&c):
  Real time             =    0.09 sec. (1.46 ticks)
Sequential b&c:
  Real time             =   33.38 sec. (40958.32 ticks)
                          ------------
Total (root+branch&cut) =   33.47 sec. (40959.78 ticks)
setting c0 = 0.0 to ensure that intercept is not penalized
02/15/20 @ 08:07 PM | 183 rows in lookup table
02/15/20 @ 08:07 PM | ------------------------------------------------------------
02/15/20 @ 08:07 PM | runnning initialization procedure
02/15/20 @ 08:07 PM | ------------------------------------------------------------
02/15/20 @ 08:07 PM | CPA produced 2 cuts
02/15/20 @ 08:07 PM | running naive rounding on 3 solutions
02/15/20 @ 08:07 PM | best objective value: 0.6931
02/15/20 @ 08:07 PM | rounding produced 3 integer solutions
02/15/20 @ 08:07 PM | best objective value is 0.6931
02/15/20 @ 08:07 PM | running sequential rounding o

...

02/15/20 @ 08:19 PM | best objective value: 0.6931
02/15/20 @ 08:19 PM | rounding produced 3 integer solutions
02/15/20 @ 08:19 PM | best objective value is 0.6931
02/15/20 @ 08:19 PM | running sequential rounding on 3 solutions
02/15/20 @ 08:19 PM | best objective value: 0.6931
02/15/20 @ 08:19 PM | sequential rounding produced 0 integer solutions
02/15/20 @ 08:19 PM | polishing 3 solutions
02/15/20 @ 08:19 PM | best objective value: 0.6931
02/15/20 @ 08:19 PM | polishing produced 3 integer solutions
02/15/20 @ 08:19 PM | initialization produced 5 feasible solutions
02/15/20 @ 08:19 PM | best objective value: 0.4255
02/15/20 @ 08:19 PM | ------------------------------------------------------------
02/15/20 @ 08:19 PM | completed initialization procedure
02/15/20 @ 08:19 PM | ------------------------------------------------------------
02/15/20 @ 08:19 PM | 183 rows in lookup table
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPX

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/15/20 @ 08:19 PM | adding 235 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4255.
Tried aggregator 1 time.
Reduced MIP has 28 rows, 30 columns, and 81 nonzeros.
Reduced MIP has 13 binaries, 15 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/15/20 @ 08:19 PM | adding 206 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4251.
Tried aggregator 1 time.
Reduced MIP has 28 rows, 30 columns, and 81 nonzeros.
Reduced MIP has 13 binaries, 15 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 218499 10668        cutoff              0.4123        0.4094   908895    0.70%          rho_12 D 218499 218498     33
 223703  8732        cutoff              0.4123        0.4099   923032    0.57%          rho_11 D 223703 223702     24
 228582  6331        cutoff              0.4123        0.4105   935951    0.43%           rho_3 D 228582 228580     20
 233100  3252        0.4115    13        0.4123        0.4115   948316    0.20%           rho_7 D 233100 233099     24

User cuts applied:  369

Root node processing (before b&c):
  Real time             =    0.09 sec. (1.51 ticks)
Sequential b&c:
  Real time             =   37.55 sec. (45557.58 ticks)
                          ------------
Total (root+branch&cut) =   37.64 sec. (45559.09 ticks)
setting c0 = 0.0 to ensure that intercept is not penalized
02/15/20 @ 08:20 PM | 183 rows in lookup table
02/15/20 @ 08:20 PM | ------------------------------------------------------------
02/15/20 @ 08:20 PM | runnning initialization procedure

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
02/15/20 @ 08:20 PM | adding 220 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4254.
Tried aggregator 1 time.
Reduced MIP has 28 rows, 30 columns, and 81 nonzeros.
Reduced MIP has 13 binaries, 15 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

In [14]:
np.mean(riskslim_summary['KY_validation']), np.mean(riskslim_summary['FL_score'])

(0.6747312226928392, 0.5434814620940537)

In [16]:
#### save results
summary_misdemeanor2_ky_inter_model = {"cart": cart_summary,
                                       "ebm": ebm_summary,
                                       "stumps": stumps_summary,
                                       "riskslim": riskslim_summary}

In [17]:
path = "./results/interpretable/two-year/"
results = [["Misdemeanor",             
            np.str(round(np.mean(cart_summary['FL_score']),3)) + " (" + np.str(round(np.std(cart_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(ebm_summary['FL_score']), 3)) + " (" + np.str(round(np.std(ebm_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(stumps_summary['FL_score']),3)) + " (" + np.str(round(np.std(stumps_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(riskslim_summary['FL_score']), 3)) + " (" + np.str(round(np.std(riskslim_summary['FL_score']),3)) + ")"]]
with open(path + 'two-year-ky-interpretable.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)